![GMV](https://www.gmv.com/export/system/modules/com.gmv.teresa.site/resources/theme/img/logo_gmv.svg)  ![Apache Spark](http://spark.apache.org/images/spark-logo.png)

# KDD99 Unsupervised Learning

# Apache Spark Initialization

In [ ]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="SecurityDataScience")

## 0. Libraries

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import * 
sqlContext = SQLContext(sc)

## 1. Data Description

**Intrinsic attributes**

These attributes are extracted from the headers' area of the network packets.

Col|Feature name  | description |	type
---|--------------|-------------|------------
1  |duration 	  |length (number of seconds) of the connection |continuous
2  |protocol_type |type of the protocol, e.g. tcp, udp, etc. |discrete
3  |service 	  |network service on the destination, e.g., http, telnet, etc. |discrete
4  |flag 	      |normal or error status of the connection. The possible status are this: SF, S0, S1, S2, S3, OTH, REJ, RSTO, RSTOS0, SH, RSTRH, SHR 	|discrete 
5  |src_bytes 	  |number of data bytes from source to destination 	|continuous
6  |dst_bytes 	  |number of data bytes from destination to source 	|continuous
7  |land 	      |1 if connection is from/to the same host/port; 0 otherwise 	|discrete
8  |wrong_fragment|sum of bad checksum packets in a connection 	|continuous
9  |urgent 	      |number of urgent packets. Urgent packets are packets with the urgent bit activated 	|continuous


**Class attribute**

The 42nd attribute is the ***class_attack*** attribute, it indicates which type of connections is each instance: normal or which attack. The values it can take are the following: *anomaly, dict, dict_simple, eject, eject-fail, ffb, ffb_clear, format, format_clear, format-fail, ftp-write, guest, imap, land, load_clear, loadmodule, multihop, perl_clear, perlmagic, phf, rootkit, spy, syslog, teardrop, warez, warezclient, warezmaster, pod, back, ip- sweep, neptune, nmap, portsweep, satan, smurf and normal*.

** Categories of class attribute **


class_attack |Category
-------|--------------
smurf| dos
neptune| dos
back| dos
teardrop| dos
pod| dos
land| dos
normal|normal
satan|probe
ipsweep|probe
portsweep|probe
nmap|probe
warezclient|r2l
guess_passwd|r2l
warezmaster|r2l
imap|r2l
ftp_write|r2l
multihop|r2l
phf|r2l
spy|r2l
buffer_overflow|u2r
rootkit|u2r
loadmodule|u2r
perl|u2r

## 2. Load Data

In [ ]:
textFileConn = sc.textFile('./data/KDD/KDDTrain+.txt', 4)


In [ ]:
#Creating the schema

#we define the name of the columns

columnNames=["class_attack", "duration","protocol_type","service","flag","src_bytes","dst_bytes","land",
                 "wrong_fragment","urgent"]

In [ ]:
#quick fields initialitation all for FloatType
connFields = [StructField(colName, FloatType(), True) for colName in columnNames]

In [ ]:
#we proceed to modify the respective fields so that they reflect the correct data type:
connFields[0].dataType = StringType()
connFields[2].dataType = StringType()
connFields[3].dataType = StringType()
connFields[4].dataType = StringType()

In [ ]:
# we can construct our schema, which we will use later below for building the data frame
connSchema = StructType(connFields)

In [ ]:
#Parsing the file
def parseReg(p):
    return ( p[41]
            ,float(p[0])
            ,p[1], p[2], p[3] 
            ,float(p[4])
            ,float(p[5])
            ,float(p[6])
            ,float(p[7])
            ,float(p[8])
            )

In [ ]:
connParsedFile = (textFileConn.map(lambda line: line.split(','))
                              .map(parseReg))

In [ ]:
# We are now ready to build our data frame, using the connParsedFile RDD computed above and the schema 
# variable already calculated:
conn = sqlContext.createDataFrame(connParsedFile, connSchema)
conn.cache()

In [ ]:
conn.take(3)

In [ ]:
conn.limit(4).toPandas()

In [ ]:
#get all the distint values of class_attack
conn.select("class_attack").distinct().toPandas()

## 3. Data Preparation

### 3.1 Encoding categorical features

In [ ]:
from pyspark.sql import functions as F

In [ ]:
def encodeCategorical(df, catName):
    #Encode the categorical variable in different columns foreach categories 
    #and the value is equal to 1 if the category is equal to column name and 0 otherwise. 
    #Finally drops the categorical variable
    
    categories = df.select(catName).distinct().toPandas()[catName]
    aux = df
    for c in categories:
        aux = aux.withColumn(c, F.when(df[catName] == c, 1).otherwise(0))
        
    return aux.drop(catName)

### Encoding *protocol_type*

In [ ]:
conn.select("protocol_type").distinct().toPandas()

In [ ]:
connEncoded = encodeCategorical(conn, "protocol_type")

In [ ]:
connEncoded.limit(10).toPandas()

### Encoding *service*

In [ ]:
connEncoded.select("service").distinct().toPandas()

In [ ]:
connEncoded = encodeCategorical(connEncoded, "service")

In [ ]:
connEncoded.limit(10).toPandas()

### Encoding *flag*

In [ ]:
connEncoded.select("flag").distinct().toPandas()

In [ ]:
connEncoded = encodeCategorical(connEncoded, "flag")

In [ ]:
connEncoded.limit(10).toPandas()

###  Encoding *class_attack* (**label**) like Integers

In [ ]:
connEncoded.select("class_attack").distinct().toPandas()

In [ ]:
categories = connEncoded.select("class_attack").distinct().toPandas()["class_attack"]

In [ ]:
dictCategories = dict((v,int(k)) for (k,v) in categories.to_dict().items())

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
def categoriesToInt(cat):
    return dictCategories[cat]

udfCategoriesToInt = udf(categoriesToInt, IntegerType())

In [ ]:
connEncoded = connEncoded.withColumn("class_attack", udfCategoriesToInt("class_attack") )

In [ ]:
connEncoded.limit(10).toPandas()

### 3.2 Input Normalization

http://spark.apache.org/docs/latest/api/python/pyspark.ml.html#module-pyspark.ml.feature

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import DenseVector

In [ ]:
connEncoded.limit(5).toPandas()

In [ ]:
features = connEncoded.drop("class_attack")

In [ ]:
features.limit(10).toPandas()

In [ ]:
stats = features.describe().toPandas()

In [ ]:
minValue = np.array(stats[stats.summary=="min"].values[0][1:], float)

In [ ]:
maxValue = np.array(stats[stats.summary=="max"].values[0][1:], float)

In [ ]:
def minMaxScaler(minV, maxV, row):
    return DenseVector([(row[i]-minV[i])/(maxV[i]-minV[i]) for i in range(len(row))])
    

In [ ]:
labeledData = connEncoded.rdd.map(lambda x: (x[0], minMaxScaler(minValue, maxValue, x[1:])))
# Apache Spark 1.6.0
#labeledData = connEncoded.map(lambda x: (x[0], minMaxScaler(minValue, maxValue, x[1:])))

In [ ]:
labeledDataFrame = sqlContext.createDataFrame(labeledData, ["label", "features"])

In [ ]:
labeledDataFrame.limit(5).toPandas()

### 3.3 Principal Component Analysis (PCA)

In [ ]:
from pyspark.ml.feature import PCA

In [ ]:
v = labeledDataFrame.limit(1).toPandas()["features"].values[0]
n_features = len(v.array)
print("Total number of features: %d" %n_features)

In [ ]:
def estimateCovariance(df):
    """Compute the covariance matrix for a given dataframe.

    Note:
        The multi-dimensional covariance array should be calculated using outer products.  Don't
        forget to normalize the data by first subtracting the mean.

    Args:
        df:  A Spark dataframe with a column named 'features', which (column) consists of DenseVectors.

    Returns:
        np.ndarray: A multi-dimensional array where the number of rows and columns both equal the
            length of the arrays in the input dataframe.
    """
    m = df.select(df['features']).rdd.map(lambda x: x[0]).mean()
    dfZeroMean = df.select(df['features']).rdd.map(lambda x:   x[0]).map(lambda x: x-m)  # subtract the mean

    return dfZeroMean.map(lambda x: np.outer(x,x)).sum()/df.count()

In [ ]:
from numpy.linalg import eigh

def pca(df, k=2):
    """Computes the top `k` principal components, corresponding scores, and all eigenvalues.

    Note:
        All eigenvalues should be returned in sorted order (largest to smallest). `eigh` returns
        each eigenvectors as a column.  This function should also return eigenvectors as columns.

    Args:
        df: A Spark dataframe with a 'features' column, which (column) consists of DenseVectors.
        k (int): The number of principal components to return.

    Returns:
        tuple of (np.ndarray, RDD of np.ndarray, np.ndarray): A tuple of (eigenvectors, `RDD` of
        scores, eigenvalues).  Eigenvectors is a multi-dimensional array where the number of
        rows equals the length of the arrays in the input `RDD` and the number of columns equals
        `k`.  The `RDD` of scores has the same number of rows as `data` and consists of arrays
        of length `k`.  Eigenvalues is an array of length d (the number of features).
     """
    cov = estimateCovariance(df)
    col = cov.shape[1]
    eigVals, eigVecs = eigh(cov)
    inds = np.argsort(eigVals)
    eigVecs = eigVecs.T[inds[-1:-(col+1):-1]]  
    components = eigVecs[0:k]
    eigVals = eigVals[inds[-1:-(col+1):-1]]  # sort eigenvals
    score = df.select(df['features']).rdd.map(lambda x: x[0]).map(lambda x: np.dot(x, components.T) )
    # Return the `k` principal components, `k` scores, and all eigenvalues

    return components.T, score, eigVals

In [ ]:
comp, score, eigVals = pca(labeledDataFrame)

In [ ]:
varianceExplained = eigVals.cumsum()/eigVals.sum()

In [ ]:
varianceExplained

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(1 - varianceExplained, drawstyle = 'steps-post')
plt.title('PCA Reconstruction Error');

In [ ]:
n_factors = ((1 - varianceExplained) > 0.10).sum()
print("Number of factors with 10% of reonstraction Error: ", n_factors)

#### 3.2.1 Apache Spark Implementation

In [ ]:
#Apache Spark API
pca = PCA(k = n_factors, inputCol="features", outputCol="pca_features")

In [ ]:
labeledDataFrame

In [ ]:
pcaModel = pca.fit(labeledDataFrame)

In [ ]:
pcaDataFrame = pcaModel.transform(labeledDataFrame).drop("features")

In [ ]:
pcaDataFrame.limit(10).toPandas()

### 3.4 Sampling

In [ ]:
# Sampling with replacement with the 30% of data
trainingData = pcaDataFrame.sample(withReplacement = True, fraction = 0.30).cache()

In [ ]:
trainingData.limit(10).toPandas()

## 4. Modeling

## 4.1 Cluster metrics

Two desirable objectives for any cluster assignment:
* **homogeneity**: each cluster contains only members of a single class.
* **completeness**: all members of a given class are assigned to the same cluster.

The main cluster metrics are:

* **Homogeneity Score**: A clustering result satisfies homogeneity if all of its clusters contain only data points which are members of a single class.
    * Bounded scores: 0.0 is as bad as it can be, 1.0 is a perfect score

* **Completeness Score**: A clustering result satisfies completeness if all the data points that are members of a given class are elements of the same cluster.
    * Bounded scores: 0.0 is as bad as it can be, 1.0 is a perfect score

* **V measure Scores** : the harmonic mean between homogeneity and completeness: v = 2 * (homogeneity * completeness) / (homogeneity + completeness)
    * Bounded scores: 0.0 is as bad as it can be, 1.0 is a perfect score

## 4.2 K-means

* *k:* number of desired clusters.
* *maxIter:* maximum number of iterations to run.
* *initMode:* specifies either random initialization or initialization via k-means.
* *initSteps:* determines the number of steps in the k-means|| algorithm.
* *tol:* determines the distance threshold within which we consider k-means to have converged.
* initialModel:* an optional set of cluster centers used for initialization. If this parameter is supplied, only one run is performed.

In [ ]:
from pyspark.ml.clustering import KMeans

### 4.2.1 Random centroid initialization (5 clusters)

In [ ]:
kmeans = KMeans(k = 5, initMode = "random", initSteps = 10, 
                featuresCol = "pca_features", maxIter = 10, tol = 1e-3 )

In [ ]:
model = kmeans.fit(trainingData)

In [ ]:
clusterCenters = model.clusterCenters()

In [ ]:
pd.DataFrame(clusterCenters)

In [ ]:
predictedData = model.transform(pcaDataFrame)
predictedData.cache()

In [ ]:
predictedData.limit(10).toPandas()

In [ ]:
# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(row, centers):
    center = centers[row["prediction"]]
    point = row["pca_features"]
    return np.sqrt(np.sum([x**2 for x in (point - center)]))

In [ ]:
print("KMeans error:" , predictedData.rdd.map(lambda row: error(row, clusterCenters)).reduce(lambda a,b : a+b))

In [ ]:
kmeansMeasures = predictedData.groupBy(predictedData.label, predictedData.prediction).count().toPandas()

In [ ]:
#completeness (all members of a given class are assigned to the same cluster)
print("Completeness:" , 1.0*np.sum(kmeansMeasures.groupby("label")["count"].max())/np.sum(kmeansMeasures["count"]))

In [ ]:
kmeansMeasures[kmeansMeasures.label==3]

In [ ]:
#homogeneity (each cluster contains only members of a single class.)
print("Homogeneity:", 1.0*np.sum(kmeansMeasures.groupby("prediction")["count"].max())/np.sum(kmeansMeasures["count"]))

In [ ]:
kmeansMeasures[kmeansMeasures.prediction==2]

### 4.2.2 Finding the optimal number of clusters

In [ ]:
def kmeansClusterMeasures(n_Clusters, dataFrame):
    #build a kmean cluster from dataFrame and with n_Clusters centroides
    #and returns the error, completeness and homogeneity of kmean's model

    trainingData, testData = dataFrame.randomSplit([70.0, 30.0])
    
    kmeans = KMeans(k = n_Clusters, initMode = "random", initSteps = 10, 
                    featuresCol = "pca_features", maxIter = 10, tol = 1e-3 )
    model = kmeans.fit(trainingData)
    centers = model.clusterCenters()
    
    predictedData = model.transform(testData)
    predictedData.cache()
    
    errorKmeans =  predictedData.rdd.map(lambda row: error(row, centers)).reduce(lambda a,b : a+b)
    
    kmeansMeasures = predictedData.groupBy(predictedData.label, predictedData.prediction).count().toPandas()
    completeness = 1.0*np.sum(kmeansMeasures.groupby("label")["count"].max())/np.sum(kmeansMeasures["count"])
    homogeneity = 1.0*np.sum(kmeansMeasures.groupby("prediction")["count"].max())/np.sum(kmeansMeasures["count"])
    
    
    return n_Clusters, errorKmeans, completeness, homogeneity

In [ ]:
def findBestCluster(minNClusters, maxNClusters, dataFrame):
    nClusters = np.arange(minNClusters, maxNClusters+1)
    kmeansMeasures = [kmeansClusterMeasures(n, trainingData) for n in nClusters]
    return pd.DataFrame(kmeansMeasures, 
                        columns = ["nClusters", "error", "completeness", "homogeneity"],
                       index = nClusters)

In [ ]:
%%time
print(kmeansClusterMeasures(5, trainingData))

In [ ]:
%%time
kmeansMeasures = findBestCluster(4, 20, trainingData)

In [ ]:
kmeansMeasures

In [ ]:
fig, axes = plt.subplots(1, 3)
kmeansMeasures.error.plot(ax=axes[0],title="Kmeans Error", grid = True, figsize=(15,5))
axes[0].set_xlabel("Number of Clusters")
kmeansMeasures.completeness.plot(ax=axes[1],title="Completeness", grid = True)
axes[1].set_xlabel("Number of Clusters")
kmeansMeasures.homogeneity.plot(ax=axes[2],title="Homogeneity", grid = True)
axes[2].set_xlabel("Number of Clusters");

#### Create the clusters with  11 centroids

In [ ]:
%%time
kmeans = KMeans(k = 11, initMode = "random", initSteps = 10, 
                featuresCol = "pca_features", maxIter = 10, tol = 1e-3 )
model = kmeans.fit(trainingData)
clusterCenters = model.clusterCenters()

In [ ]:
%%time
predictedData = model.transform(pcaDataFrame)
predictedData.cache()

In [ ]:
predictedData.limit(5).toPandas()

#### Visualize the results on PCA-reduced data

In [ ]:
from pyspark.ml.feature import PCA
from pyspark.sql import Row

In [ ]:
pca = PCA(k=2, inputCol="pca_features", outputCol="pca_2D")
modelPCA = pca.fit(predictedData)
projectedDT = modelPCA.transform(predictedData)

In [ ]:
projectedDT.limit(5).toPandas()

In [ ]:
clusterCentersDF = sc.parallelize(
    [Row(n, DenseVector(cluster.tolist())) for n, cluster in zip(range(len(clusterCenters)), clusterCenters)]
).toDF(["prediction", "pca_features"])

In [ ]:
clusterCentersDF.toPandas()

In [ ]:
clusterCenters2D = modelPCA.transform(clusterCentersDF)

In [ ]:
centers2D = clusterCenters2D.rdd.map(lambda row: (row["prediction"], row["pca_2D"]))\
                            .map(lambda v: Row(prediction = v[0], x1 = v[1].values.tolist()[0],\
                                                     x2 = v[1].values.tolist()[1]))\
                            .toDF().toPandas()

In [ ]:
n_points = 10000
points2D = projectedDT.limit(n_points).rdd\
                      .map(lambda row: (row["label"], row["prediction"], row["pca_2D"]))\
                      .map(lambda v: Row(label = v[0], 
                                                prediction = v[1], 
                                                x1 = v[2].values.tolist()[0], 
                                                x2 = v[2].values.tolist()[1]))\
                       .toDF().toPandas()

In [ ]:
points2D.head()

In [ ]:
# set colors

from matplotlib import colors
import six
colors_ = list(six.iteritems(colors.cnames))

# Add the single letter colors.
for name, rgb in six.iteritems(colors.ColorConverter.colors):
    hex_ = colors.rgb2hex(rgb)
    colors_.append((name, hex_))

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(14, 7) )
ax1, ax2 = axes.ravel()

# Plot the clusters
for i, color in zip(set(points2D.prediction), colors_):
    idx = np.where(points2D.prediction == i)
    ax1.scatter(points2D.values[idx, 2], points2D.values[idx, 3], c=color, s = 10, label=i, alpha = 0.25, edgecolors='none')
    # Plot the centroids as X
    ax1.scatter(centers2D.values[i, 1], centers2D.values[i, 2],
            marker='x', s=169, linewidths=3,
            color=color, zorder=10)
ax1.set_title('K-mean prediction - Random centroid initialization (11 clusters)')
ax1.legend()



#Plot the category values
for i, color in zip(set(points2D.label), colors_):
    idx = np.where(points2D.label == i)
    ax2.scatter(points2D.values[idx, 2], points2D.values[idx, 3], c=color, s = 10, label=i, alpha = 0.25, edgecolors='none')
    
ax2.set_title('Category Attacks (11 categories)')
ax2.legend();

#### Visualize the results on PCA-reduced data 3D

In [ ]:
pca = PCA(k=3, inputCol="pca_features", outputCol="pca_3D")
modelPCA = pca.fit(predictedData)
projectedDT = modelPCA.transform(predictedData)

In [ ]:
projectedDT.limit(5).toPandas()

In [ ]:
clusterCentersDF = sc.parallelize(
    [Row(n, DenseVector(cluster.tolist())) for n, cluster in zip(range(len(clusterCenters)), clusterCenters)]
).toDF(["prediction", "pca_features"])

In [ ]:
clusterCentersDF.toPandas()

In [ ]:
clusterCenters3D = modelPCA.transform(clusterCentersDF)

In [ ]:
centers3D = clusterCenters3D.rdd.map(lambda row: (row["prediction"], row["pca_3D"]))\
                            .map(lambda v: Row(prediction = v[0], 
                                                       x1 = v[1].values.tolist()[0], 
                                                       x2 = v[1].values.tolist()[1],
                                                       x3 = v[1].values.tolist()[2]))\
                            .toDF().toPandas()

In [ ]:
centers3D

In [ ]:
n_points = 1000
points3D = projectedDT.limit(n_points).rdd\
                      .map(lambda row: (row["label"], row["prediction"], row["pca_3D"]))\
                      .map(lambda v: Row(label = v[0], 
                                                prediction = v[1], 
                                                x1 = v[2].values.tolist()[0], 
                                                x2 = v[2].values.tolist()[1],
                                                x3 = v[2].values.tolist()[2]))\
                       .toDF().toPandas()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(8,8))
ax = fig.gca(projection='3d')

# Plot the clusters
for i, color in zip(set(points3D.prediction), colors_):
    idx = np.where( points3D.prediction == i)
    ax.scatter(points3D.values[idx, 2], points3D.values[idx, 3], points3D.values[idx, 4],
               c=color, label=i, s=50, alpha = 0.3, 
               edgecolors='none')
    # Plot the centroids as X
    ax.scatter(centers3D.values[i, 1], centers3D.values[i, 2], centers3D.values[i, 3], 
               marker='x', linewidths=3, s = 150,
               color=color, zorder=10)
ax.set_title('K-mean prediction - Random centroid initialization (11 clusters)')
ax.legend();